使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第十一章第一節 地域の区分と気象条件）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import numpy as np
import pandas as pd
from numpy import exp, log
from functools import lru_cache


## 第一節　地域の区分と外気条件 / Section 1 Regional division and ambient conditions

### 1．適用範囲 / Scope of application

省略

ommision

### 2. 引用規格 / Cited standard

省略

ommision

### 3. 用語の定義 / Definition of terms

省略

ommision

### 4. 外気条件 / Outside air condition

省略

ommision

### 5. 地域の区分 / Segmentation of area

省略

ommision

## 付録 A 外気相対湿度の計算方法  / Appendix A How to calculate ambient relative humidity

### A.1 記号及び単位 / Symbol and unit

省略

ommision

### A.2 外気相対湿度の計算方法 / How to calculate ambient relative humidity

In [ ]:
@lru_cache()
def load_outdoor():
    """気温[℃],絶湿[g/kg']

    :return: 気温[℃],絶湿[g/kg']
    :rtype: DateFrame
    """
    return pd.read_csv('data/outdoor.csv', skiprows=4, nrows=24 * 365, names=(
    'day', 'hour', 'holiday', 'Theta_ex_1', 'X_ex_1', 'Theta_ex_2', 'X_ex_2', 'Theta_ex_3', 'X_ex_3', 'Theta_ex_4',
    'X_ex_4', 'Theta_ex_5', 'X_ex_5', 'Theta_ex_6', 'X_ex_6', 'Theta_ex_7', 'X_ex_7', 'Theta_ex_8', 'X_ex_8'))


In [ ]:
@lru_cache()
def load_climate(region):
    return pd.read_csv('data/climate/climateData_{}.csv'.format(region), nrows=24 * 365, encoding="SHIFT-JIS")


In [ ]:
# 日射量
def get_J(climate):
    return climate["水平面天空日射量 [W/m2]"].values


In [ ]:
def get_Theta_ex(region, df):
    """気温[℃]

    :param region: 省エネルギー地域区分
    :type region: int
    :param df: 気温[℃],絶湿[g/kg']
    :type df: DateFrame
    :return: 気温[℃]
    :rtype: ndarray
    """
    return df['Theta_ex_' + str(region)].values


In [ ]:
def get_X_ex(region, df):
    """絶湿[g/kg']

    :param region: 省エネルギー地域区分
    :type region: int
    :param df: 気温[℃],絶湿[g/kg']
    :type df: DateFrame
    :return: 絶湿[g/kg']
    :rtype: ndarray
    """
    return df['X_ex_' + str(region)].values

$$\large
h_{ex} = \frac{P_{v}}{P_{vs}}\times 100
$$

$h_{ex}$：外気相対湿度(%) / Relative humidity  
$P_{v}$：外気の水蒸気圧(Pa) / Water vapor pressure of outside air    
$P_{vs}$：外気の飽和水蒸気圧(Pa) / Saturated water vapor pressure of outside air  

In [ ]:
def calc_h_ex(X_ex, Theta_ex):
    """外気相対湿度 式（１）

    :param X_ex: 絶湿[g/kg']
    :type X_ex: ndarray
    :param Theta_ex: 外気絶対温度[K]
    :type Theta_ex: ndarray
    :return: 外気相対温度
    :rtype: ndarray
    """
    P_v = get_P_v(X_ex)
    P_vs = calc_P_vs(Theta_ex)
    return P_v / P_vs * 100


$$\large
P_{v} = 101325 \times \frac{X_{ex}}{(622+X_{ex})}
$$

$X_{ex}$：外気絶対湿度(g/kg') / Absolute air humidity  

In [ ]:
def get_P_v(X_ex):
    """外気の水蒸気圧　式（２）

    :param X_ex: 絶湿[g/kg']
    :type X_ex: ndarray
    :return: 外気の水蒸気圧
    :rtype:ndarray
    """
    return 101325 * (X_ex / (622 + X_ex))


$$\large
P_{vs} = e^{k}
$$

In [ ]:
def calc_P_vs(Theta_ex):
    """外気の飽和水蒸気圧　式（3a）

    :param Theta_ex: 外気絶対温度[K]
    :type Theta_ex: ndarray
    :return: 外気の飽和水蒸気圧
    :rtype: ndarray
    """
    k = get_k(Theta_ex)
    return exp(k)


$$\large
\displaystyle
k = \left\{ \begin{array}{ll}
    \frac{a_{1}}{T_{ex}}+a_{2}+a_{3}\times T_{ex}+a_{4}\times T_{ex}^{2}+a_{5}\times log_{e}(T_{ex}) & (\theta_{ex} > 0)\\
    \frac{b_{1}}{T_{ex}}+b_{2}+b_{3}\times T_{ex}+b_{4}\times T_{ex}^{2}+b_{5}\times log_{e}(T_{ex}) & (\theta_{ex} \leq 0)\\
    \end{array}
\right.
$$

$T_{ex}$：外気絶対温度(K) / Absolute temperature  
$\theta_{ex}$：外気温度(℃) / Outside air temperature  

<div style="text-align: center;font-weight: bold;">表　式における係数の値<br>Table: Value of coefficient in formula</div>

<table>
    <thead>
<tr><th>係数<br>coefficient</th><th>値<br>value</th></tr>
    </thead>
    <tbody>
<tr><th>$a_{1}$</th><td>－6096.9385</td></tr>
<tr><th>$a_{2}$</th><td>21.2409642</td></tr>
<tr><th>$a_{3}$</th><td>－0.02711193</td></tr>
<tr><th>$a_{4}$</th><td>0.00001673952</td></tr>
<tr><th>$a_{5}$</th><td>2.433502</td></tr>
<tr><th>$b_{1}$</th><td>－6024.5282</td></tr>
<tr><th>$b_{2}$</th><td>29.32707</td></tr>
<tr><th>$b_{3}$</th><td>0.010613863</td></tr>
<tr><th>$b_{4}$</th><td>－0.000013198825</td></tr>
<tr><th>$b_{5}$</th><td>－0.49382577</td></tr>
    </tbody>
</table>


In [ ]:
def get_k(Theta_ex):
    """指数k 式(3b)

    :param Theta_ex: 外気絶対温度[K]
    :type Theta_ex: ndarray
    :return: 指数k
    :rtype: ndarray
    """
    T_ex = get_T_ex(Theta_ex)

    a1 = -6096.9385
    a2 = 21.2409642
    a3 = -0.02711193
    a4 = 0.00001673952
    a5 = 2.433502
    b1 = -6024.5282
    b2 = 29.32707
    b3 = 0.010613863
    b4 = -0.000013198825
    b5 = -0.49382577

    # Theta_ex > 0
    k_a = a1 / T_ex + a2 + a3 * T_ex + a4 * T_ex ** 2 + a5 * log(T_ex)

    # Theta_ex <= 0
    k_b = b1 / T_ex + b2 + b3 * T_ex + b4 * T_ex ** 2 + b5 * log(T_ex)

    return k_a * (Theta_ex > 0) + k_b * (Theta_ex <= 0)


$$\large
T_{ex} = \theta_{ex}+273.16
$$

In [ ]:
def get_T_ex(Theta_ex):
    """外気絶対温度[K]

    :param Theta_ex: 気温[℃]
    :type Theta_ex: ndarray
    :return: 外気絶対温度[K]
    :rtype:  ndarray
    """
    return Theta_ex + 273.16


## 付録 B 平均外気温度の計算方法  / Appendix B

### B.1 記号及び単位 / Symbol and unit

省略

### B.2 年平均外気温度

$$\large 
\theta_{ex,\alpha\_Ave} = \sum^{365}_{d = 1} \sum^{23}_{t = 0} \theta_{ex,d,t} \big/ 8760
$$

$\theta_{ex,\alpha\_Ave}$：年平均外気温度(℃)  
$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  

In [ ]:
def get_Theta_ex_a_Ave(Theta_ex_d_t):
    """年平均外気温度[℃] (1)

    :param Theta_ex_d_t: 外気絶対温度[K]
    :type Theta_ex_d_t: ndarray
    :return: 年平均外気温度[℃]
    :rtype: ndarray
    """
    return np.sum(Theta_ex_d_t) / 8760

### B.3 日平均外気温度 / Daily average outside air temperature

$$\large 
\theta_{ex,Ave,d} = \sum^{23}_{t = 0} \theta_{ex,d,t} \big/ 24
$$

$ \theta_{ex,Ave,d} $：日付$d$における日平均外気温度(℃) / Daily average outside temperature on date $ d $  
$ \theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃) / Outside temperature at time $ t $ of date $ d $  

In [ ]:
def get_Theta_ex_d_Ave_d(Theta_ex_d_t):
    """日付dにおける日平均外気温度[℃] (2)

    :param Theta_ex_d_t: 外気絶対温度[K]
    :type Theta_ex_d_t: ndarray
    :return: 日付dにおける日平均外気温度
    :rtype: ndarray
    """

    # 1次元配列を2次元配列に形状変換する
    Theta_ex_d_t = np.reshape(Theta_ex_d_t, (365, 24))

    # 時間軸を合計し、365日分の配列を返す
    return np.sum(Theta_ex_d_t, axis=1) / 24

### B.4 暖房期における期間平均外気温度

$$\large 
\theta_{ex,H\_Ave} = \frac{\sum_{d \in D} \sum^{23}_{t = 0} \theta_{ex,d,t}}{24\times|D|}
$$

$$\large 
D: = \{x|0 < \small\sum^{23}_{t = 0} \sum^{12}_{i = 1} \large L_{H,x,t,i} \}
$$

$D$：暖房負荷が発生する日付の集合(-)  
$\theta_{ex,H\_Ave}$：暖房期における期間平均外気温度(℃)  
$\theta_{ex,d,t}$：日付$d$における外気温度(℃)  

In [ ]:
def get_Theta_ex_H_Ave(Theta_ex_d_t, L_H_x_t_i):
    """暖房期における期間平均外気温度[℃] (3a)

    :param Theta_ex_d_t: 外気絶対温度[K]
    :type Theta_ex_d_t: ndarray
    :param L_H_x_t_i: 暖冷房区画iの1時間当たりの暖房負荷
    :type L_H_x_t_i: ndarray
    :return: 暖房期における期間平均外気温度[℃]
    :rtype: ndarray
    """

    # 暖房負荷が発生する日付の集合[-]
    D = get_D(L_H_x_t_i)

    # 1次元配列を2次元配列に形状変換する
    Theta_ex_d_t = np.reshape(Theta_ex_d_t, (365, 24))

    # 時間軸合算
    Theta_ex_d = np.sum(Theta_ex_d_t, axis=1)

    # 暖房期以外の外気温度を0とする
    Theta_ex_d[D == False] = 0

    # 暖房期における期間平均外気温度[℃]
    Theta_ex_H_Ave = np.sum(Theta_ex_d) / (24 * np.count_nonzero(D))

    return Theta_ex_H_Ave

In [ ]:
def get_D(L_H_x_t_i):
    """ 暖房負荷が発生する日付の集合[-] (3b)

    :param L_H_x_t_i: 暖冷房区画iの1時間当たりの暖房負荷
    :type L_H_x_t_i: ndarray
    :return: 暖房負荷が発生する日付の集合
    :rtype: ndarray
    """

    # L_H_x_t_iは暖冷房区画毎に365日×24時間分の負荷を持った2次元配列
    # 暖冷房区画軸合算(暖冷房区画の次元をなくす)
    L_H_x_t = np.sum(L_H_x_t_i, axis=0)

    # 1次元配列を2次元配列に形状変換する
    L_H_x_t = np.reshape(L_H_x_t, (365, 24))

    # 時間軸合算
    L_H_x = np.sum(L_H_x_t, axis=1)

    # 暖房日の判定（暖房負荷がある日はtrue、ない日はfalse）
    D = 0 < L_H_x

    # 暖房日の判定を行った365日分の配列を返す
    return D